# 05: Duality and Arrangements

*Authors: Felix Espey*

This notebook serves as supplementary learning material for the course **Geometric Algorithms**.
It showcases and explains implementations of algorithms presented in the corresponding lecture, and elaborates on some practical considerations concerning their use.
Furthermore, it offers interactive visualisations and animations.

## Table of Contents

1. Introduction  
2. Setup
3. Duality  
    3.1. Setup  
    3.2. Visualisation  
    3.3. Algorithm(s)
4. Arragements
5. References

## 1. Introduction

This chapter contains two connected topics, **Duality** and **Arragements**.

**Duality** is an alternative approach to think about points and lines, representing one as the other. This allows to use algorithms for one problem on one on the problems for the other. For example, checking if 3 points are on a line is the same as checking if the dual lines of those points all intersect in a single point.

An **Arangement** $\mathcal{A}$ is a subdivision of the plane induced by a set $L$ of lines. It has vertices (intersection of the lines of $L$), edges (segments of the lines of $L$) and faces (area inbetween the lines of $L$).

Many problems on a set of points can be solved by calculating the Arrangement on the dual lines of those points.

We will first go over duality and some examples to make the concept more clear before going to Arangements later.

PLACEHOLDER IMAGE

<img style='float: left;' src='./images/04-image00.png' width='440px' height='412'>
<img style='float: left;' src='./images/04-image01.png' width='440px' height='412'>

## 2. Setup

First let's do some setup. This is not very interesting, so you can skip to Section 3 if you want.

We now import everything we'll need throughout this notebook from external sources, including our module for generic data structures, our module for geometric primitives and operations as well as our module for visualisation purposes. 
These modules are explained in [notebook no. 00](./00-Basics.ipynb).

## 3. Duality

In the intro I already explained how **Duality** alows us to represent points and lines and vice versa. This could in theory be done in any convoluted way we wanted, but here we have a simple transformation.

A **point** $p = (x, y)$ becomes the line $p* := (y = p_x x - p_y)$. $p*$ is then called the dual of $p$.
You can see that the x-coordinate of the point becomes the slope of the line, while the y-coordinate becomes the offset of the line in the y direction. This also lets us define the inverse pretty easily.

A **line** $l = mx + b$ becomes the point $l* = (m, -b)$. The slope becomes the x-coordinate and the y-offset negated becomes the y-coordinate.

A **Line Segment** defined by two points $p_1, p_2$ becomes the area between the two dual lines of those points. 

The concept itself is pretty easy to understand, but since it can be hard to visualise. To make this easier, we will create a visual aid to play around with below

### 3.1 Setup

We start by setting up some things like imports and the visualisation tool. This is not relevant to the lecture, so you can freely skip this section.

In [3]:
from modules.geometry import Point, Line, LineSegment

### 3.2 Visualisation

Next we have to set up some functions to use in the visualisation. First lets make some simple functions to convert points and lines into the respective duals.

In [4]:
def getDualFromPoint(p : Point) -> Line:
    return Line(p.x, p.y)

def getDualFromLine(l : Line) -> Point:
    return Point(l.p1, -l.p2)

def getDualFromLineSegment(ls : LineSegment) -> tuple[Line,Line]:
    return tuple[getDualFromPoint(ls.upper), getDualFromLineSegment(ls.lower)]

## 4 Arrangements

An **Arrangement** $\mathcal{A}(L)$ is a subdivision of the plane induced by a set $L$ of lines. It has vertices (intersection of the lines of $L$), edges (segments of the lines of $L$) and faces (area inbetween the lines of $L$).

The outer edges and faces of an Arrangement are unbounded. For edges this happens because the lines that make the edge have some last intersection with the other lines of $L$. The edge after that intersection has no other endpoint. The outer faces are also unbounded as they have no additional line limiting their size.

Since DCLEs can only store bounded edges and faces, we generate a bounding box $\mathcal{B}$ around $L$ and only consider the parts of lines inside this box. This adds additional vertices where the lines intersect with the bounding box. We also get new edges as the boundary of the box becomes part of the DCEL.

Once we have a DCLE form the bounding box we can add the lines of $L$ incrementally. We do this by following along the line from left to right, splitting each face we find on the way by adding new Edges and Vertices as needed. 

The steps to computing the Arrangement $\mathcal{A}(L)$ then are as follows:
1. compute the bounding box $\mathcal{B}(L)$
2. initialize DCEL with the edges, vertices and interior face from the bounding box
3. incrementally add the lines of $L$ into the DCEL
   
We will go over these steps one by one with a more detailed explanation for step 3. as it is the main part of the algorithm.

### 4.1 Bounding box

As stated above, the first step is to calculate the bounding box $\mathcal{B}$ of the set of lines $L$.

This bounding box should contain all vertices of the Arrangment $\mathcal{A}(L)$ of $L$. Since the vertices of $\mathcal{A}(L)$ are just the intersection points of the lines in $L$, what we actually need to find are the maximum and minimum x and y coordinate of the intersections. Then we can use them as cornerpoints for an axis alinged box that contains all vertices.

To find the intersection we points we could use a linesweep algorithm, but in this case we will just go with a bruteforce Algorithm that runs in quadratic time. In theory this is super easy, but in pratice we need to be careful with special cases. The steps are as follows:

1. First we need to find all Intersections between the lines. For this we just use a bruteforce approach by checking all possible combinations of lines
2. Search the list of intersections for the biggest and smallest x and y coordinates.
3. Build the corners of the bounding box from the min/max x/y coordinates of the intersection points

Since it is possible that two lines intersect very far away, we will ignore any intersections outside visible area. Since we only check for intersections within the bounding box when calculating the arrangement, this does not change the algorithm. It does however change the resulting arragement itself.

In [1]:
from modules.geometry.core import Rectangle, Point, Line
from itertools import combinations
from modules.geometry import PointSequence
from modules.visualisation import VisualisationTool, LineSetInstance
from modules.visualisation.drawing import PointsMode

def computeBoundingBox(lines:set[Line], visibleArea : Rectangle) -> PointSequence:
    points : PointSequence = PointSequence()
    botLeft = Point(float('inf'), float('inf'))
    topRight = Point(float('-inf'), float('-inf'))
    points.append(botLeft)
    points.append(topRight)
    for line1, line2 in combinations(lines, 2):
        intersection = line1.getIntersection(line2)
        if(type(intersection) is Point and visibleArea.isInside(intersection)):
            points.append(intersection)
            if(intersection.x < botLeft.x):
                newBotLeft = Point(intersection.x, botLeft.y)
                points.update(botLeft, newBotLeft)
                botLeft = newBotLeft
            if(intersection.y < botLeft.y):
                newBotLeft = Point(botLeft.x, intersection.y)
                points.update(botLeft, newBotLeft)
                botLeft = newBotLeft
            if(intersection.x > topRight.x):
                newtopRight = Point(intersection.x, topRight.y)
                points.update(newtopRight, newBotLeft)
                topRight = newtopRight
            if(intersection.y > topRight.y):
                newtopRight = Point(topRight.x, intersection.y)
                points.update(topRight, newtopRight)
                topRight = newtopRight
            points.delete(intersection)
    return points

def computeBoundingBoxAlg(lines:set[Line]):
    computeBoundingBox(lines, Rectangle(Point(10,10), Point(390,390)))

p1 = Point(10, 10)
p2 = Point(100, 1000)
p3 = Point(100, 20)
p4 = Point(300, 900)
p5 = Point(1000, 100)

lines = [Line(p1,p2 ), Line(p1,p5 ), Line(p3,p4), Line(p2,p4)]

print(computeBoundingBoxAlg(lines))

#visualisation_tool = VisualisationTool(400, 400, LineSetInstance())
#canvas_size = min(visualisation_tool.width, visualisation_tool.height)
#visualisation_tool.register_algorithm("boundingBox", computeBoundingBoxAlg, PointsMode())
#visualisation_tool.display()

AttributeError: 'PointSequence' object has no attribute 'update'

In [1]:
from modules.geometry import Point, PointReference, Orientation, PointExtension, PointList as ORT
from modules.visualisation import PointsMode, VisualisationTool, PointSetInstance, LineSetInstance
from modules.visualisation.drawing import LineMode

def makePoints(points: set[Point]):
    pass

class test1:
    def execute_on(points: list[Point]):
        pass

class test2(test1):
    def execute_on(points: list[PointExtension]):
        return super().execute_on()

list : list[PointExtension] = []
test2.execute_on(list)


visualisation_tool = VisualisationTool(400, 400, LineSetInstance())
canvas_size = min(visualisation_tool.width, visualisation_tool.height)
visualisation_tool.register_algorithm("makePoints", makePoints, LineMode())


ModuleNotFoundError: No module named 'dcel_animator'

In [2]:
visualisation_tool.display()